In [1]:
import xarray as xr
import metpy.calc as mpcalc
from metpy import units
import cartopy.crs as ccrs, cartopy.feature as cf
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
import pandas as pd
from netCDF4 import Dataset
from wrf import getvar

In [2]:
lonW = -100
lonE = -25
latS = -60
latN = 0
cLat, cLon = (latS + latN)/2, (lonW + lonE)/2
proj_map = ccrs.Robinson(central_longitude=cLon)
proj_data = ccrs.PlateCarree()
res = '10m'

In [3]:
ds1 = xr.open_dataset('REFL_interpolated.nc')#.isel(Time=slice(36,38))
ds2 = xr.open_dataset('500_REFL_interpolated.nc')#.isel(Time=slice(36,38))
ds3 = xr.open_dataset('500_WRF_interpolated.nc')#.isel(Time=slice(36,38))
ds4 = xr.open_dataset('SFC_interpolated.nc')#.isel(Time=slice(36,38))

In [4]:
ds1

<xarray.Dataset>
Dimensions:           (south_north: 285, west_east: 393, Time: 97, level: 2)
Coordinates:
    XLONG             (south_north, west_east) float32 ...
    XLAT              (south_north, west_east) float32 ...
    XTIME             (Time) float64 ...
  * Time              (Time) datetime64[ns] 2018-12-13T10:00:00 ... 2018-12-1...
    datetime          (Time) datetime64[ns] ...
  * level             (level) int64 1000 4000
Dimensions without coordinates: south_north, west_east
Data variables:
    REFL_10CM_interp  (Time, level, south_north, west_east) float32 ...

In [5]:
lon, lat, time = ds1.XLONG, ds1.XLAT, ds1.XTIME

In [6]:
# remember to edit
startYear = 2018
startMonth = 12
startDay = 13
startHour = 10
startMinute = 0
startDateTime = dt(startYear,startMonth,startDay, startHour, startMinute)

endYear = 2018
endMonth = 12
endDay = 14
endHour = 10
endMinute = 0
endDateTime = dt(endYear,endMonth,endDay, endHour, endMinute)

In [7]:
dateList = pd.date_range(startDateTime, endDateTime, freq='.25H')
dateList

DatetimeIndex(['2018-12-13 10:00:00', '2018-12-13 10:15:00',
               '2018-12-13 10:30:00', '2018-12-13 10:45:00',
               '2018-12-13 11:00:00', '2018-12-13 11:15:00',
               '2018-12-13 11:30:00', '2018-12-13 11:45:00',
               '2018-12-13 12:00:00', '2018-12-13 12:15:00',
               '2018-12-13 12:30:00', '2018-12-13 12:45:00',
               '2018-12-13 13:00:00', '2018-12-13 13:15:00',
               '2018-12-13 13:30:00', '2018-12-13 13:45:00',
               '2018-12-13 14:00:00', '2018-12-13 14:15:00',
               '2018-12-13 14:30:00', '2018-12-13 14:45:00',
               '2018-12-13 15:00:00', '2018-12-13 15:15:00',
               '2018-12-13 15:30:00', '2018-12-13 15:45:00',
               '2018-12-13 16:00:00', '2018-12-13 16:15:00',
               '2018-12-13 16:30:00', '2018-12-13 16:45:00',
               '2018-12-13 17:00:00', '2018-12-13 17:15:00',
               '2018-12-13 17:30:00', '2018-12-13 17:45:00',
               '2018-12-

In [8]:
refl = ds1['REFL_10CM_interp'].sel(level=4000)
refl500 = ds2['REFL_10CM_interp'].sel(level=4000)

In [9]:
reflLevs = np.arange(0, 70, 5)

In [10]:
wrf_file = Dataset('/nfs/lazearlab_rit/lucy/3km_500/500_wrfout_files/wrfout_d02_2018-12-14_01:15:00')
ter500 = getvar(wrf_file, "ter", timeidx=-1)
wrf_file = Dataset('/nfs/lazearlab_rit/lucy/3km_morrison_ysu_control/control_wrfout_files/wrfout_d02_2018-12-14_01:15:00')
terctl = getvar(wrf_file, "ter", timeidx=-1)

In [11]:
u10_500 = ds3['u10m']
v10_500 = ds3['v10m']
u10 = ds4['u10m']
v10 = ds4['v10m']

In [12]:
from wrf import to_np

In [ ]:
i = 0
for value in time:
    fig = plt.figure(figsize=(18,12)) 
    ax = plt.subplot(1,1,1,projection=proj_map)
    ax.set_extent ([lonW+31,lonE-36,latS+24.8,latN-28.5])
    ax.add_feature(cf.COASTLINE.with_scale(res))
    ax.add_feature(cf.STATES.with_scale(res))
    
    
    skip=9
    ax.barbs(to_np(lon[::skip,::skip]), to_np(lat[::skip,::skip]), to_np(u10.isel(Time=i)[::skip,::skip]), to_np(v10.isel(Time=i)[::skip,::skip]), length=6, color='orange', flip_barb=True, transform=proj_data, label='With Terrain')
    ax.barbs(to_np(lon[::skip,::skip]), to_np(lat[::skip,::skip]), to_np(u10_500.isel(Time=i)[::skip,::skip]), to_np(v10_500.isel(Time=i)[::skip,::skip]), length=6, color='blue', flip_barb=True, transform=proj_data, label='Without Terrain')
    
    
    CL1 = ax.contour(ter500.XLONG, ter500.XLAT, ter500, levels=np.arange(500, 3000, 1000), colors='blue', alpha=.6, linewidths=3, linestyles='dotted', transform=proj_data)
    ax.clabel(CL1)
    
    CL2 = ax.contour(terctl.XLONG, terctl.XLAT, terctl, levels=np.arange(500, 3000, 1000), colors='orange', alpha=.4, linewidths=3, linestyles='solid', transform=proj_data)
    ax.clabel(CL2)
    
    
    CF1 = ax.contourf(lon, lat, refl.isel(Time=i), levels=reflLevs, colors='orange', zorder=2, alpha=.8, transform=proj_data)
    
    CF2 = ax.contourf(lon, lat, refl500.isel(Time=i), levels=reflLevs, colors='blue', alpha=.4, zorder=2, transform=proj_data)
    
    timeStr = dt.strftime(dateList[i], format='%H%M UTC %d %B %Y')
    
    tl1 = 'WRF-Control and WRF-Terrain-500'
    tl2 = '4000-m Reflectivity (dBZ), Wind (kts), and Terrain Height (m)'
    tl3 = timeStr
    title_line = (tl1 + '\n' + tl2 + '\n' + tl3 + '\n')
    
    plt.title(title_line)
    
    ax.legend(loc='upper right')
    
    i = i + 1